In [1]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torch.nn as nn
from pathlib import Path
import pandas as pd
import numpy as np
import random
import torch

from library.GCN import ConvolutionLayer, PoolingLayer, GraphData, collate_graph_dataset, Standardizer, Graph
from library.cVAE import GCN_Encoder, GRU_Decoder, cVAE
from torch.utils.data import DataLoader,SubsetRandomSampler

In [2]:
# Load QM9 SMILES
df_qm9 = pd.read_pickle('../data/RDKit/rdkit_only_valid_smiles_qm9.pkl')
smiles_list = df_qm9["SMILES"].to_list()

# Collect all unique characters
charset = set()
for smi in smiles_list:
    for ch in smi:
        charset.add(ch)

# Sort for consistency
charset = sorted(list(charset))

# Add special tokens
special_tokens = ['<PAD>', '<END>', '<STR>']
vocab_list = special_tokens + charset

# Create token -> index mapping
token2idx = {tok: idx for idx, tok in enumerate(vocab_list)}
idx2token = {idx: tok for tok, idx in token2idx.items()}

print("Vocabulary size:", len(vocab_list))
print("Example tokens:", vocab_list)

Vocabulary size: 24
Example tokens: ['<PAD>', '<END>', '<STR>', '#', '(', ')', '+', '-', '/', '1', '2', '3', '4', '5', '=', '@', 'C', 'F', 'H', 'N', 'O', '[', '\\', ']']


In [ ]:
def padding():
    pass

In [3]:
def train_model(
    epoch,
    model,
    training_dataloader,
    optimizer,
    loss_fn,
    use_GPU,
    max_atoms,
    node_vec_len,
):
    """
    Custom function which defines how a model will be trained (per epoch), here the mean-squared loss between prediction and actual value is used as evaluation metric. This function will perform backpropagation which updates the weights of the networks based in this evaluation.
    """
    # Create variables to store losses and error
    avg_loss = 0
    count = 0

    # Switch model to train mode
    model.train()

    # Go over each batch in the dataloader
    for i, dataset in enumerate(training_dataloader):
        # Unpack data
        node_mat = dataset[0][0]
        adj_mat = dataset[0][1]
        gap = dataset[1]

        smiles = []
        for smile in dataset[2]:
            char_list = ['<STR>'] + list(smile) + ['<END>']
            vocab_idx_list = torch.as_tensor([token2idx[ch] for ch in char_list])
            smiles.append(vocab_idx_list)

        # ---- Pad sequences ----
        # find max length
        max_seq_len = max(s.size(0) for s in smiles)

        # pad with a PAD token index
        PAD_IDX = token2idx['<PAD>']

        padded_smiles = torch.full((len(smiles), max_seq_len), PAD_IDX, dtype=torch.long)

        for i, seq in enumerate(smiles):
            padded_smiles[i, :seq.size(0)] = seq
        
        batch_size, _ = padded_smiles.size()

        # Reshape inputs
        node_mat = node_mat.reshape(batch_size, max_atoms, node_vec_len)
        adj_mat = adj_mat.reshape(batch_size, max_atoms, max_atoms)

        # Package inputs and outputs; check if GPU is enabled
        if use_GPU:
            model_input = (node_mat.cuda(), adj_mat.cuda(), padded_smiles, gap.cuda())
            model_output = padded_smiles
        else:
            model_input = (node_mat, adj_mat, padded_smiles, gap)
            model_output = padded_smiles

        # Compute output from network
        model_prediction_distribution = model(*model_input) # [batch_size, max_smiles_seq_len, vocab_size]

        # Calculate loss
        loss = loss_fn(model, model_prediction_distribution.permute(0, 2, 1), model_output, batch_size)
        avg_loss += loss

        # Calculate MAE
        # prediction = standardizer.restore(nn_prediction.detach().cpu())
        # mae = mean_absolute_error(output, prediction)
        # avg_mae += mae

        # Set zero gradients for all tensors
        optimizer.zero_grad()

        # Do backward prop
        loss.backward()

        # Update optimizer parameters
        optimizer.step()

        # Increase count
        count += 1

    # Calculate avg loss and MAE
    avg_loss = avg_loss / count
    # avg_mae = avg_mae / count

    # Print stats
    # print(
    #     "Epoch: [{0}]\tTraining Loss: [{1:.2f}]\tTraining MAE: [{2:.2f}]"\
    #        .format(
    #                 epoch, avg_loss, avg_mae
    #        )
    # )

    print(
        "Epoch: [{0}]\tTraining Loss: [{1:.2f}]".format(epoch, avg_loss)
    )

    # Return loss and MAE
    return avg_loss #, avg_mae

In [4]:
#### Inputs
n_epochs = 3
batch_size = 1000
train_size = 0.7
learning_rate = 0.01
device = "cpu"

# GCN
max_atoms = 30 # fixed value
node_vec_len = 16 # fixed value
n_features = 32
n_conv_layers = 2
n_hidden_layers = 2

# GRU
latent_dim = 16
gru_dim = 16
embedding_dim = 8
n_layers = 2

# cVAE
vocab_size = len(vocab_list)
gcn_hidden_nodes = n_features + 1
teacher_forcing_ratio = 0.5

In [5]:
#### Start by creating dataset
main_path = Path.cwd().parents[0]
data_path = main_path / "data" / "RDKit" / "rdkit_only_valid_smiles_qm9.pkl"
dataset = GraphData(dataset_path=data_path, max_atoms=max_atoms, node_vec_len=node_vec_len)


#### Split data into training and test sets
# Get train and test sizes
dataset_indices = np.arange(0, len(dataset), 1)
train_size = int(np.round(train_size * len(dataset)))
test_size = len(dataset) - train_size

# Randomly sample train and test indices
train_indices = np.random.choice(dataset_indices, size=train_size, 
                                                            replace=False)
test_indices = np.array(list(set(dataset_indices) - set(train_indices)))

# Create dataoaders
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                          sampler=train_sampler, 
                          collate_fn=collate_graph_dataset)
test_loader = DataLoader(dataset, batch_size=batch_size, 
                         sampler=test_sampler,
                         collate_fn=collate_graph_dataset)

In [6]:
encoder = GCN_Encoder(
    node_vec_len=node_vec_len,
    node_fea_len=n_features,
    hidden_fea_len=n_features,
    n_conv=n_conv_layers,
    n_hidden=n_hidden_layers,
    n_outputs=1,
    p_dropout=0.1
)

decoder = GRU_Decoder(
    vocab_size=vocab_size,
    latent_dim=latent_dim,
    property_dim=1,
    hidden_size=gru_dim,
    n_layers=n_layers,
    embedding_dim=embedding_dim
).to(device)

model = cVAE(
    encoder=encoder,
    decoder=decoder,
    device=device,
    n_gcn_hidden_dim=gcn_hidden_nodes,
    n_gru_hidden_dim=gru_dim,
    latent_dim=latent_dim,
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
).to(device)

In [7]:
# Standardizer
# smiles = [dataset[i][2] for i in range(len(dataset))]

# standardizer = Standardizer(torch.Tensor(outputs))

In [8]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
def loss_function(model, logits, targets, batch_size, beta=1):
    recon_loss_fn = nn.CrossEntropyLoss(ignore_index=0)
    loss_recon = recon_loss_fn(logits, targets)
    
    kl_loss = -0.5 * torch.sum(1 + model.z_logvar - model.z_mean.pow(2) - model.z_logvar.exp()) / batch_size
    loss = loss_recon + beta * kl_loss

    return loss

In [10]:
use_GPU = False

#### Train the model
loss = []
epoch = []
for i in range(n_epochs):
    epoch_loss = train_model(
        i,
        model,
        train_loader,
        optimizer,
        loss_function,
        use_GPU,
        max_atoms,
        node_vec_len,
    )
    loss.append(epoch_loss)
    epoch.append(i)

Epoch: [0]	Training Loss: [2.29]
Epoch: [1]	Training Loss: [1.47]
Epoch: [2]	Training Loss: [1.32]


Need to add:
- a form of accuracy
- CV for hyperparameter tuning